In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import NuestrasFunciones

from names_dataset import NameDataset
nd = NameDataset()
from countryinfo import CountryInfo


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, scale
import statsmodels.api as sm 

In [4]:
df_titanic = pd.read_csv("In/train.csv")

In [5]:
df_titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
def return_float_list(df):
    lista_float = []
    ser_dtypes = df.dtypes
    for i,val in enumerate(ser_dtypes):
        if val == "float":
            lista_float.append(ser_dtypes.index[i])
    return lista_float
    
return_float_list(df_titanic)

['Age', 'Fare']

In [7]:
def limpia(df):
    df.rename(columns = {"SibSp": "SiblingsSpouses","Parch":"ParentsChildren"},inplace = True)
    #df.drop("PassengerId",axis = 1,inplace = True)

    df.loc[df["Sex"] == "male", "Sex"] = 1
    df.loc[df["Sex"] == "female", "Sex"] = 0
    df["Sex"] =  df["Sex"].astype("category")

        
    return df       

In [8]:
def cant_mismo(tick,ser):
    lista = []
    val_counts = ser.value_counts()
    valor = val_counts[tick] -1
    return valor


In [9]:
def Operaciones_Ticekt(df):
    x = df['Ticket'].apply(lambda x: x.split(" "))
    
    df["PostTicket"] = [i[1] if len(i) == 2 else i[0] for i in x]
    df.Ticket = df.Ticket.apply(lambda x: x.replace(",","").replace(".","").replace("/","").upper())
    df["Cant_PersonasMismoTicket"] = df.Ticket.apply(lambda x: cant_mismo(x,df.Ticket))


    df["PreTicket"] = [i[0] if len(i) == 2 else "desconocido" for i in x]
    df.PreTicket = df.PreTicket.apply(lambda x: x.replace(",","").replace(".","").replace("/","").upper())
    df.PreTicket = df.PreTicket.apply(lambda x: x[0] if x[0].lower() != "s" else x[0:2])
    #df_titanic.PreTicket = df_titanic.PreTicket.apply(lambda x: x if x in ["SC","SO"] else x[0])
    df.PreTicket = df.PreTicket.apply(lambda x: x if x == "SO" else x[0])
    df.PreTicket = df.PreTicket.apply(lambda x: "DESC" if x == "D" else x)
    
    return df


In [10]:
def disgrega_titulo(df):
    df = df.copy()
    df["Titulo_Expandido"] = df["Titulo"]
    for i in df.Titulo_Expandido:
        #print(i)
        if i in["Mr","Mrs","Miss","Master"]:
            
            df.loc[((df["ParentsChildren"] == 0) & (df["Titulo_Expandido"] == i)),"Titulo_Expandido"] = f"{i}_SinParentsChildren"
            df.loc[((df["ParentsChildren"] != 0) & (df["Titulo_Expandido"] == i)),"Titulo_Expandido"] = f"{i}_ConParentsChildren"
        
        
            df.loc[((df["Cantidad_MismoApellido"] == 0) & (df["Titulo_Expandido"] == f"{i}_SinParentsChildren")),"Titulo_Expandido"] = f"{i}_SinParentsChildren_SinPersMismoApe"
            df.loc[((df["Cantidad_MismoApellido"] != 0) & (df["Titulo_Expandido"] == f"{i}_SinParentsChildren")),"Titulo_Expandido"] = f"{i}_SinParentsChildren_ConPersMismoApe"
            df.loc[((df["Cantidad_MismoApellido"] == 0) & (df["Titulo_Expandido"] == f"{i}_ConParentsChildren")),"Titulo_Expandido"] = f"{i}_ConParentsChildren_SinPersMismoApe"
            df.loc[((df["Cantidad_MismoApellido"] != 0) & (df["Titulo_Expandido"] == f"{i}_ConParentsChildren")),"Titulo_Expandido"] = f"{i}_ConParentsChildren_ConPersMismoApe"
        
        
            df.loc[((df["SiblingsSpouses"] != 0) & (df["Titulo_Expandido"] == f"{i}_SinParentsChildren_SinPersMismoApe")),"Titulo_Expandido"] = f"{i}_ConSoloHermanastros"
    return df


In [11]:
def return_pais(ape):

    try:
        dict = nd.search(ape)["last_name"]["country"]
        maximo = max(zip(dict.values(),dict.keys()))[1]
    except TypeError:
        try:
           dict = nd.search(ape)["first_name"]["country"]
           maximo = max(zip(dict.values(),dict.keys()))[1]
        except TypeError:
            maximo = "Desconodido"
    return maximo

def get_language(pais):
    try:
        idioma = CountryInfo(pais).languages()[0]
    except:
        return "Desc"
    return idioma 


In [12]:
def Operaciones_Nombres(df):
    
    x = df.Name.apply(lambda x: x.split(","))
    df["Apellido"] = df.Name.apply(lambda x: x.split(",")[0])
    df["Cantidad_MismoApellido"] = df["Apellido"].apply(lambda x: cant_mismo(x,df.Apellido))

    df["Titulo"] = df.Name.apply(lambda x: x.split(",")[1].split(".")[0].replace(" ",""))

    df = disgrega_titulo(df)

    df["Pais"] = df.Apellido.apply(lambda x: return_pais(x))  
    df["Idioma"] = df.Pais.apply(lambda x: get_language(x))
    return df



In [13]:
def feat_eng(df):
    df = Operaciones_Ticekt(df)
    df = Operaciones_Nombres(df)        
    return df

#### El test también tiene valores missing

# Te falta conceptualizar los imputadores y arreglar bugs

In [14]:
def Imputa_cabin(df):
    #Fit
    df_ApeCabin = df[["Apellido","Cabin"]].copy().dropna()#.reset_index(drop = True)
    df_TicketCabin = df[["PostTicket","Cabin"]].copy().dropna()#.reset_index(drop = True)
    
    Apellidos_ConCabin = list(df_ApeCabin.Apellido.values)
    Tickets_ConCabin = list(df_TicketCabin.PostTicket.values)
    #Transform
    for idx,(ape, cabina,tick) in enumerate(zip(df.Apellido,df.Cabin,df.PostTicket)):
        
        if cabina is np.nan:
            if ape in Apellidos_ConCabin:
                #print(ape)
                df.loc[idx, "Cabin"] = df_ApeCabin.loc[df["Apellido"] == ape, "Cabin"].values[0]
            elif tick in Tickets_ConCabin:
                df.loc[idx, "Cabin"] = df_TicketCabin.loc[df["PostTicket"] == tick, "Cabin"].values[0]
            else:
                df.loc[idx, "Cabin"] = "desconocido"

    df["Cabin_Letra"] = df.Cabin.apply(lambda x: x[0] if x != "desconocido" else "desc")
    return df

def imputa_age(df, titulo_expanido = False):

    if titulo_expanido:
        #Fit
        llave = df[["Age","Titulo_Expandido"]]
        llave = llave.groupby("Titulo_Expandido").agg("mean").reset_index()
        #Transform
        for idx, val in enumerate(df.Titulo_Expandido):
            if np.isnan(df.Age[idx]):
                df.loc[idx, "Age"] = llave.loc[llave["Titulo_Expandido"] == val, "Age"].values[0]
    
    else:
        df_agg = df[["Age","Titulo"]]
        llave = df_agg.groupby("Titulo").agg("mean").reset_index()
        for idx, val in enumerate(df.Titulo):
            if np.isnan(df.Age[idx]):
                df.loc[idx, "Age"] = llave.loc[llave["Titulo"] == val, "Age"].values[0]
    return df

### Aplicando lo aprendido en hands on machine learning de Aurelién Gerón, al imputar el test se hará con los datos del train.

In [15]:
#Imputamos cabin asignando misma cabin a personas del mismo apellido, en su defecto ticket, en su defecto desconocido.
#Imputamos Age por edad promedio del título o del título disgregado por vínculos familiares a bordo.
#Imputamos Cabin por más frecuente por país.
class Imputador(BaseEstimator, TransformerMixin):

    def __init__(self, titulo_expandido = False):
        self.titulo_expandido = titulo_expandido  
        
    def fit(self, X, y=None):
        #CABIN
        
        self.df_ApeCabin = X[["Apellido","Cabin"]].dropna().copy()#.reset_index(drop = True)
        self.df_TicketCabin = X[["PostTicket","Cabin"]].dropna().copy()#.reset_index(drop = True)
        self.Apellidos_ConCabin = list(self.df_ApeCabin.Apellido.values)
        self.Tickets_ConCabin = list(self.df_TicketCabin.PostTicket.values)
        
        #AGE
        if self.titulo_expandido:
            self.llave_titulo = X[["Age","Titulo_Expandido"]]
            self.llave_titulo = self.llave_titulo.groupby("Titulo_Expandido").agg("mean").reset_index()
        else: 
            self.llave_titulo = X[["Age","Titulo"]]
            self.llave_titulo = self.llave_titulo.groupby("Titulo").agg("mean").reset_index()

        #Embarked tiene un par de missings, imputaré por moda por País.
        self.df_embarked  = X[["Pais","Embarked"]].dropna().copy().groupby("Pais").apply(lambda x: x.mode().iloc[0]).reset_index(drop=True)

        #Fare
        self.llave_Fare = X[["Fare","Titulo_Expandido"]]
        self.llave_Fare = self.llave_Fare.groupby("Titulo_Expandido").agg("mean").reset_index()


        return self 
    
    def transform(self, X, y=None): 
        #CABIN
        for idx,(ape, cabina,tick) in enumerate(zip(X.Apellido,X.Cabin,X.PostTicket)):
            if cabina is np.nan:
                if ape in self.Apellidos_ConCabin:
                    #X.loc[idx, "Cabin"] = 4
                    #zz  = self.df_ApeCabin.loc[self.df_ApeCabin["Apellido"] == ape, "Cabin"].values[0]
                    X.loc[idx, "Cabin"] =  self.df_ApeCabin.loc[self.df_ApeCabin["Apellido"] == ape, "Cabin"].values[0]
                    
                    #X.loc[idx, "Cabin"] = self.df_ApeCabin.loc[X["Apellido"] == ape, "Cabin"].values[0]
                    
                elif tick in self.Tickets_ConCabin:
                    X.loc[idx, "Cabin"] = self.df_TicketCabin.loc[self.df_TicketCabin["PostTicket"] == tick, "Cabin"].values[0]
                    
                else:
                    X.loc[idx, "Cabin"] = "desconocido"
    
        X["Cabin_Letra"] = X.Cabin.apply(lambda x: x[0] if x != "desconocido" else "desc")

        #AGE
        for idx, (val, val_exp) in enumerate(zip(X.Titulo, X.Titulo_Expandido)):
            #print("-------------------------------------------------")
            #print(self.llave_titulo)
            #print("el error?")
            #print(X.Age)
            
            if self.titulo_expandido:
                if np.isnan(X.Age[idx]):
                    X.loc[idx, "Age"] = self.llave_titulo.loc[self.llave_titulo["Titulo_Expandido"] == val_exp, "Age"].values[0]
        
            else:
                if np.isnan(X.Age[idx]):
                    X.loc[idx, "Age"] = self.llave_titulo.loc[self.llave_titulo["Titulo"] == val, "Age"].values[0]


        #Embarked
        for idx,(val_embark,val_pais) in enumerate(zip(X.Embarked,X.Pais)):
            if val_embark is np.nan:
                X.loc[idx, "Embarked"] = self.df_embarked.loc[self.df_embarked["Pais"] == val_pais, "Embarked"].values[0]


        #Fare
        for idx, val_titulo in enumerate(X.Titulo_Expandido):
            if np.isnan(X.Fare[idx]):

                X.loc[idx, "Fare"] = self.llave_Fare.loc[self.llave_Fare["Titulo_Expandido"] == val_titulo, "Fare"].values[0]

               
                

        return X



        

In [16]:
def rank_columna_fit(df,columna):
    #fit
    # Creo title ranks del df antes de sub-categorizar los títulos por si alguna de las subcategorías no aparece en el test
    df_agg = df[[columna,"Survived"]].copy()
    df_agg = df_agg.groupby([columna], as_index = False).agg("mean").sort_values(by = "Survived")

    return df_agg

In [17]:
def rank_columna_transform(df, columna, df_agg):
    arr_rank = []
    for i in df[columna]:
        
        if i in list(df_agg[columna]):
            arr_rank.append(df_agg.loc[df_agg[columna] == i, "Survived"].values[0])

        else:
            arr_rank.append(0.383838)

    #Cambio el valor numérico por rankings del 1 al 19 para que el valor de la variable no sea combinación lineal directa de Y.
    # Algoritmo que agarra el df sorted by Titulo_Rank y le asigna una categoría cardinal a cada score.
    columna += "_Rank"
    df[columna] = pd.Series(arr_rank)
    df = df.sort_values(by = columna, inplace = False).reset_index(drop = True)
    
    arr_intermed = list(df[columna])
    #Todos los scores iguales a w tendrán el mismo ranking, cuando el score no sea igual a w, se actualiza el ranking a asignar.
    w = arr_intermed[0]
    j = 1
    for i in range(len(df[columna])):
        if  arr_intermed[i] == w:
            arr_intermed[i] = j
        else:
            w = arr_intermed[i]
            j += 1
            arr_intermed[i] = j

    df[columna] = pd.Series(arr_intermed)
        
    return df    

In [18]:
#Asigno rankings a ciertos features del df. Esto lo hago en vez de dejar el survival rate de las personas con ese feature para
#así evitar tener features que son directamente combinaciones lineales de la objetuvo.
#Tampoco hago one hot enconding por la baja cantidad de observaciones y la pequeña muestra de ciertas categorías.
class asigna_ranks(BaseEstimator, TransformerMixin):

    def __init__(self, usa_titulo_expandido = True):
        self.usa_titulo_expandido = usa_titulo_expandido

    def fit(self, X, y=None):
        self.PreTicket_Rank = rank_columna_fit(X,"PreTicket")
        self.Title_Expandido_Rank = rank_columna_fit(X,"Titulo_Expandido")
        self.Title_Rank = rank_columna_fit(X,"Titulo")
        self.Cabin_Letra_Rank = rank_columna_fit(X,"Cabin_Letra")
        self.Idioma_Rank = rank_columna_fit(X,"Idioma")
        self.Embarked_Rank = rank_columna_fit(X,"Embarked")
        

        return self 
        
    def transform(self,X,y = None):
        X = rank_columna_transform(X,"PreTicket",self.PreTicket_Rank)
        X = rank_columna_transform(X,"Titulo",self.Title_Rank)
        X = rank_columna_transform(X,"Titulo_Expandido",self.Title_Expandido_Rank)
        X = rank_columna_transform(X,"Cabin_Letra",self.Cabin_Letra_Rank)
        X = rank_columna_transform(X,"Idioma",self.Idioma_Rank)
        X = rank_columna_transform(X,"Embarked",self.Embarked_Rank)

        return X

In [19]:
from sklearn.preprocessing import scale, StandardScaler

class myScaler(BaseEstimator,TransformerMixin): 
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None, copy=None):
        columnas_float = return_float_list(X)
        escalador = StandardScaler()
        X[columnas_float] = escalador.fit_transform(X[columnas_float])
        
        for idx,feature in enumerate(columnas_float):
            X[feature] += abs(min(X[feature]))+0.0001

        return X

In [20]:
df_titanic = pd.read_csv("In/train.csv")
pipep = Pipeline([
            ('Limpiar', FunctionTransformer(limpia)),
            ('feat_eng', FunctionTransformer(feat_eng)),
            ('imputa', Imputador()),
            ('Rank_assign', asigna_ranks()),
            ('escala', myScaler()),   
        ])


df_train = pipep.fit_transform(df_titanic.copy())

In [111]:
df_train.to_csv("Out/df_imputed_transformed_train.csv")

## Test se transforma más no se ajusta

In [112]:
df_test = pd.read_csv("In/test.csv")
df_test = pipep.transform(df_test.copy())

In [113]:
df_test.to_csv("Out/df_imputed_transformed_test.csv")

# Transformaciones de features finales:

In [35]:
def final_feat_eng(df, training):
    lista1 = ["Pclass","Sex","SiblingsSpouses","Idioma_Rank","PreTicket_Rank",
             "Titulo_Rank","Cabin_Letra_Rank","Age","Titulo_Expandido_Rank","PassengerId"]
    if training:
        lista1.append("Survived")

    df = df[lista1]
    
    df = sm.add_constant(df)
    df.Sex = df.Sex.astype("int")
    df["SiblingsSpouses_raiz4"] = df.SiblingsSpouses ** (1/4)
    df["Cabin_Letra_Rank_raiz4"] = df.Cabin_Letra_Rank ** (1/4)
    df["Titulo_Expandido_Rank_raiz4"] = df.Titulo_Expandido_Rank ** (1/4)
    df["PreTicket_Rank_Titulo_Rank"] = df.PreTicket_Rank * df.Titulo_Rank 
    df["Pclass_Sex"] = df.Pclass * df.Sex
    df["Sex_Cabin_Letra_Rank_raiz4"] = df.Sex * df.Cabin_Letra_Rank_raiz4
    df["Idioma_Rank_Titulo_Expandido_Rank_raiz4"] = df.Idioma_Rank* df.Titulo_Expandido_Rank_raiz4 
    df["Age_Titulo_Expandido_Rank"] = df.Age * df.Titulo_Expandido_Rank

    lista2 = ["const","Pclass","Sex","SiblingsSpouses","Idioma_Rank",
             "SiblingsSpouses_raiz4","PreTicket_Rank_Titulo_Rank","Pclass_Sex",
             "Sex_Cabin_Letra_Rank_raiz4","Idioma_Rank_Titulo_Expandido_Rank_raiz4","Age_Titulo_Expandido_Rank","PassengerId"]
    if training:
        lista2.append("Survived")
        
    df = df[lista2]
    df.set_index("PassengerId",inplace=True)
    df.sort_index(inplace = True)
    


    return df

In [36]:
df_titanic = pd.read_csv("In/train.csv")

pipep_final = Pipeline([
            ('pipe1', pipep),
           ('feat_eng_2', FunctionTransformer(func = final_feat_eng, kw_args={"training":True})),  
        ])

pipep_final[1].set_params(kw_args={"training":True})
df_Train_Final = pipep_final.fit_transform(df_titanic.copy())

In [37]:
df_Train_Final.to_csv("Out/df_Train_Final.csv")

In [38]:
df_Train_Final.columns

Index(['const', 'Pclass', 'Sex', 'SiblingsSpouses', 'Idioma_Rank',
       'SiblingsSpouses_raiz4', 'PreTicket_Rank_Titulo_Rank', 'Pclass_Sex',
       'Sex_Cabin_Letra_Rank_raiz4', 'Idioma_Rank_Titulo_Expandido_Rank_raiz4',
       'Age_Titulo_Expandido_Rank', 'Survived'],
      dtype='object')

In [39]:
df_Test_Final = pd.read_csv("In/test.csv")
pipep_final[1].set_params(kw_args={"training":False})
df_Test_Final = pipep_final.transform(df_Test_Final)

In [40]:
df_Test_Final.to_csv("Out/df_Test_Final.csv")

In [41]:
df_Test_Final

,const,Pclass,Sex,SiblingsSpouses,Idioma_Rank,SiblingsSpouses_raiz4,PreTicket_Rank_Titulo_Rank,Pclass_Sex,Sex_Cabin_Letra_Rank_raiz4,Idioma_Rank_Titulo_Expandido_Rank_raiz4,Age_Titulo_Expandido_Rank
PassengerId,,,,,,,,,,,
892,1.0,3,1,0,12,0.0,12,3,1.000000,16.970563,10.552768
893,1.0,3,0,1,12,1.0,48,0,0.000000,24.000000,57.580088
894,1.0,2,1,0,12,0.0,12,2,1.000000,16.970563,19.005727
895,1.0,3,1,0,11,0.0,12,3,1.000000,15.556349,8.247415
896,1.0,3,0,1,10,1.0,48,0,0.000000,20.877976,31.874937
...,...,...,...,...,...,...,...,...,...,...,...
1305,1.0,3,1,0,12,0.0,2,3,1.000000,16.970563,9.897460
1306,1.0,1,0,0,8,0.0,27,0,0.000000,13.012612,20.887964
1307,1.0,3,1,0,1,0.0,4,3,1.000000,1.414214,11.782289
